In [1]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import numpy as np
import pandas as pd

# 倒入设置参数和加载数据集函数
from Config import *




# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))
    ssCnt = {}
    for tid in D:  # 对于每一条transaction
        for can in Ck:  # 对于每一个候选项集can，检查是否是transaction的一部分 # 即该候选can是否得到transaction的支持
            flag = True
            for i in can:
                if i not in tid:
                    flag = False
                    
            if flag:
                ssCnt[can] = ssCnt.get(can, 0) + 1
                
#             if can.issubset(tid):
#                 ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    # print("ssCnt is",ssCnt)
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems  # 每个项集的支持度
        if support >= minSupport:  # 将满足最小支持度的项集，加入retList
            retList.insert(0, key)
        supportData[key] = support  # 汇总支持度数据
    return retList, supportData


def aprioriGen(Lk, k):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''
    retList = set()
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            
            L1 = Lk[i]
            L2 = Lk[j]
            cnt =0
            for m in L1:
                if m in L2:
                    cnt+=1
            if cnt == k-2:
                retList.add(Lk[i] | Lk[j])
    return retList


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    C1 = createC1(
        dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]

    D = [set(var) for var in dataSet]  # 集合化数据集
    L1, suppData = scanD(D, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素
    L = [L1]  # 最初的L1中的每个项集含有一个元素，新生成的
    # print()
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        t = time.time()
        Ck = aprioriGen(L[k - 2], k)
        print(f'gen coast:{time.time() - t:.8f}s')
        
        t = time.time()
        Lk, supK = scanD(D, Ck, minSupport) # 筛选最小支持度的频繁项集
        print(f'scan coast:{time.time() - t:.8f}s')
        # print("iter is ")
        # print(Ck)
        # print(Lk)
        # print()
        suppData.update(supK)  # 将新的项集的支持度数据加入原来的总支持度字典中
        L.append(Lk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L, suppData  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
#             print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList




In [2]:
myDat = loadDataSet()  # 导入数据集
for i in myDat:
    print(i)

(1000, 8)
['MB', 'RB', 'WB', 'male', 'group E']
['female', 'group E']
['MB', 'RB', 'WB', 'female', 'group B']
['MB', 'RB', 'WB', 'male', 'group A']
['male', 'group C']
['MB', 'RB', 'WB', 'preparation', 'male', 'group A']
['male', 'group A']
['female', 'group E']
['preparation', 'female', 'group C']
['MB', 'female', 'group B']
['MB', 'female', 'group B']
['RG', 'WG', 'female', 'group C']
['MB', 'female', 'group E']
['MB', 'RB', 'WB', 'male', 'group A']
['preparation', 'female', 'group D']
['female', 'group D']
['RB', 'male', 'group D']
['MB', 'male', 'group C']
['WG', 'preparation', 'female', 'group C']
['MB', 'preparation', 'female', 'group D']
['female', 'group D']
['male', 'group C']
['preparation', 'male', 'group D']
['RG', 'female', 'group B']
['female', 'group C']
['MG', 'preparation', 'male', 'group B']
['preparation', 'male', 'group D']
['MG', 'preparation', 'male', 'group D']
['MG', 'preparation', 'male', 'group B']
['RB', 'male', 'group D']
['MG', 'male', 'group D']
['MB', 'fe

In [3]:

import time
t = time.time()

L, suppData = apriori(myDat, 0.05)  # 选择频繁项集
# print(u"频繁项集L：", suppData)
# print(u"所有候选项集的支持度信息：", suppData)
print(f'花费的时间为:{time.time() - t:.8f}s')

14
gen coast:0.00003886s
91
scan coast:0.01385212s
gen coast:0.00029421s
137
scan coast:0.02524209s
gen coast:0.00006104s
12
scan coast:0.00256610s
gen coast:0.00000095s
0
scan coast:0.00004911s
花费的时间为:0.04526687s


In [4]:
rules = generateRules(L, suppData, minConf=0.2)
import functools

def cmp(a,b):
    if a[2] < b[2]:
        return 1
    else:
        return -1
    
    
rules = sorted(rules, key=functools.cmp_to_key(cmp))
pdData = []
for A,B,conf in rules:
    print(set(A), '-->', set(B),'置信度为', conf)
    pdData.append([set(A), set(B), conf])

    # 存入本地
pdData = pd.DataFrame(pdData)
pdData.to_csv("rule.csv", index= False)

{'WB'} --> {'MB'} 置信度为 0.9385964912280701
{'RB'} --> {'MB'} 置信度为 0.9222222222222223
{'RB'} --> {'WB'} 置信度为 0.8666666666666667
{'RB'} --> {'MB', 'WB'} 置信度为 0.8222222222222222
{'WG'} --> {'RG'} 置信度为 0.8099173553719009
{'MG', 'female'} --> {'RG', 'WG'} 置信度为 0.8088235294117646
{'WG'} --> {'female'} 置信度为 0.7768595041322315
{'RG'} --> {'female'} 置信度为 0.7727272727272726
{'RG'} --> {'WG'} 置信度为 0.7424242424242424
{'WB'} --> {'male'} 置信度为 0.7280701754385965
{'WG'} --> {'MG'} 置信度为 0.7107438016528925
{'WB'} --> {'RB'} 置信度为 0.6842105263157895
{'RB'} --> {'male'} 置信度为 0.6777777777777778
{'WB'} --> {'MB', 'male'} 置信度为 0.6666666666666666
{'WG'} --> {'RG', 'female'} 置信度为 0.6611570247933884
{'RG'} --> {'MG'} 置信度为 0.6590909090909091
{'WB'} --> {'MB', 'RB'} 置信度为 0.6491228070175438
{'MG', 'WG'} --> {'RG', 'female'} 置信度为 0.6395348837209303
{'MG', 'RG'} --> {'WG', 'female'} 置信度为 0.632183908045977
{'MG'} --> {'male'} 置信度为 0.6136363636363636
{'WG'} --> {'preparation'} 置信度为 0.6115702479338843
{'MB'} --> {'femal

In [5]:
graph ={}
points ={}
for A,B,conf in rules:
    if len(B) ==1 and len(A) ==1:
        
        # 将总置信度和作为节点的大小
        if A not in points:
            points[A] = conf
        else:
            points[A] += conf
        if B not in points:
            points[B] = conf
        else:
            points[B] += conf
            
        graph[A|B] = conf
print(len(graph), points)

37 {frozenset({'WB'}): 3.5488132094943237, frozenset({'MB'}): 6.398435286432352, frozenset({'RB'}): 3.407843137254902, frozenset({'WG'}): 4.34685542576604, frozenset({'RG'}): 3.993629476584022, frozenset({'female'}): 6.689497846239161, frozenset({'male'}): 6.805049416375516, frozenset({'MG'}): 4.955648361904066, frozenset({'preparation'}): 6.295990429606062, frozenset({'group A'}): 0.5955056179775281, frozenset({'group C'}): 3.0335360687216655, frozenset({'group B'}): 2.1521653896266897, frozenset({'group D'}): 2.5647151145831146, frozenset({'group E'}): 2.0954545454545452}


# 关联规则分析
从以上分析可以看出，当最低置信度为0.2时，总计有边37点14.理论上来说，14个点的完全无向图为91条边

In [6]:
from pyecharts import options as opts
from pyecharts.charts import Graph
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK # 不能省略，解决依赖问题


In [8]:
graph

nodes=[]
for i in points:
    a, = i
    nodes.append({'name':a, 'symbolSize':points[i] * 10})
links = []
for i in graph:
    left,right = i
#     print(i, left, right)
    links.append({'source':left,'target':right, 'value':1/graph[i]})

G = (
    Graph()
    .add(
        series_name='',
        nodes=nodes,
        links=links,
        layout="circular", # 启用环形布局
        #is_rotate_label = True, # 是否旋转标签
        repulsion=8000,  # 斥力的范围
        #设置线的弯曲程度
        linestyle_opts=opts.LineStyleOpts(curve=0.1)

        )
    .set_global_opts(title_opts=opts.TitleOpts(title="关联规则算法"))
    
)
# G.load_javascript()
G.render_notebook()